In [1]:
import os
import torch
from PIL import Image
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import numpy as np
from skimage import io, transform
import matplotlib.pyplot as plt
import time
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import torch.mps 
import itertools
import csv

# plt.ion() 
plt.gray()


<Figure size 640x480 with 0 Axes>

In [2]:
# %%time
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    print("MPS found")
    device = torch.device("mps")
    

MPS found


In [3]:
class FERPlusDataset(Dataset):
    """FERPlus dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.img_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_frame)

#     to access elements using the []
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

#   to create the image name
        img_name = os.path.join(self.root_dir, self.img_frame.iloc[idx, 0])

        image = io.imread(img_name)
        # image = io.imread(image)
        image = Image.fromarray(image)

        emotions = self.img_frame.iloc[idx, 2:]
        emotions = np.asarray(emotions)
        emotions = emotions.astype('float32')

        if self.transform:
            image = self.transform(image)

        sample = {'image': image, 'emotions': emotions} # a dictionary of an image with its label
        
        return sample #return a transformed image with label


In [4]:
data_transforms = {
    'train': transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=(-15, 15)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229]),
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=(48, 48)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485], std=[0.229]),
    ]),
}

In [5]:
# #     class to transform to a normalized tensor (only the image pixel value is transformed)
# class ToTensor(object):
#     """Convert ndarrays in sample to Tensors."""

#     def __call__(self, sample):
#         image, emotions = sample['image'], sample['emotions']
#         transform = transforms.ToTensor()

#         return {'image': transform(image),
#                 'emotions': emotions}

In [6]:
train_folder_path = './data/FER2013Train'
test_folder_path = './data/FER2013Test'
valid_folder_path = './data/FER2013Valid'

In [7]:
train_dataset = FERPlusDataset(csv_file= os.path.join(train_folder_path,"label.csv"), root_dir=train_folder_path, transform=data_transforms['train'])
valid_dataset = FERPlusDataset(csv_file= os.path.join(valid_folder_path, "label.csv"), root_dir= valid_folder_path, transform=data_transforms['valid'])
test_dataset = FERPlusDataset(csv_file= os.path.join(test_folder_path, "label.csv"), root_dir= test_folder_path, transform=data_transforms['valid'])

In [8]:
train_dataset[0]


{'image': tensor([[[-2.1179, -2.1179, -2.1179,  ...,  0.4337,  0.4679, -2.1179],
          [ 0.2796,  0.0912, -0.0801,  ...,  0.4679,  0.4679, -2.1179],
          [ 0.2967, -0.0287, -0.0458,  ...,  0.4851,  0.4679, -2.1179],
          ...,
          [-2.1179,  1.0331,  1.0331,  ...,  0.0912,  1.2385,  1.0844],
          [-2.1179,  1.0673,  1.0502,  ...,  1.2557,  1.0502,  1.0331],
          [-2.1179,  1.0844,  0.9988,  ..., -2.1179, -2.1179, -2.1179]]]),
 'emotions': array([6., 0., 1., 1., 0., 0., 0., 0., 2., 0.], dtype=float32)}

In [9]:
def train_and_validate(epochs, optimizer, scheduler ,criterion, model, trainloader, validloader, learning_rate,):
    train_loss = []
    train_accuracy = []
    valid_loss = []
    valid_accuracy = []
    opt_name = optimizer.__name__
    optimizer = optimizer(model.parameters(), lr=learning_rate)
    
    if scheduler == optim.lr_scheduler.ReduceLROnPlateau:
        scheduler = scheduler(optimizer)
        # print("plateu")
        # print(type(scheduler))
        
    elif scheduler == optim.lr_scheduler.ExponentialLR: 
        scheduler = scheduler(optimizer, gamma=0.9)
        # print(type(scheduler))
        
    st = time.time()

# Training - Validation loop
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        model.train()
        # Perform training
        for i, data in enumerate(trainloader, 0):
            labels = data['emotions'].to(device)
            inputs = data['image'].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            # Calculate and store training accuracy
            _, predicted = torch.max(outputs, 1)
            _, labels = torch.max(labels, 1)
            # for performance, you can max in data prep for consistency, 
            total += inputs.size(0)
            correct +=  (predicted == labels).sum().item()
        scheduler.step()
        train_loss.append(running_loss / len(trainloader))
        train_accuracy.append(100 * correct / total)
        # Perform validation
        model.eval()
        correct = 0
        total = 0
        running_loss = 0.0
        with torch.inference_mode():
            for data in validloader:
                labels = data['emotions'].to(device)
                inputs = data['image'].to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                running_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                _, labels = torch.max(labels, 1)
                total += inputs.size(0)
                correct +=  (predicted == labels).sum().item() #can be torch.eq(pred, labels).sum().item()
        
        valid_loss.append(running_loss / len(validloader))
        valid_accuracy.append(100 * correct / total)
        
        # Print the training and validation loss and accuracy
        print(f'Epoch {epoch+1}/{epochs}:')
        print(f'Training Loss: {train_loss[-1]:.4f} | Training Accuracy: {train_accuracy[-1]:.2f}%')
        print(f'Validation Loss: {valid_loss[-1]:.4f} | Validation Accuracy: {valid_accuracy[-1]:.2f}%')
        print('-----------------------------------')

    # elapsed_time = time.time() - st
    print('Finished Training')
    # with open(f'{outputs_path}/statistics.csv', 'a', newline='') as csvfile:
    #     writer = csv.writer(csvfile)

    #     if csvfile.tell() == 0:
    #         # Write the column headers
    #         writer.writerow(['trial', 'Batch size', 'Epochs', 'Activation function', 'Loss function', 'Initial Learning rate', 'Optimizer', 'Scheduler',
    #                         'Min training accuracy', 'Max training accuracy', 'Average training accuracy',
    #                         'Min validation accuracy', 'Max validation accuracy', 'Average validation accuracy',
    #                         'Total time', 'device'])

In [10]:
# Define parameter grids
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD
activation = F.relu
learning_rate = 0.01
epochs = 20
batch_size = 32
scheduler = optim.lr_scheduler.ExponentialLR
device = torch.device("mps")

In [11]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [12]:
class Net(nn.Module):
        def __init__(self, drop=0.2):
            super().__init__()
            self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5) 
        # output size = 6 *44*44 values 
        # image size : n*n 
        # filter size: f*f (f is odd number)
        # shrinked_image size : (n - f + 1)^2 

            self.bn1 = nn.BatchNorm2d(6)  # Batch normalization after conv1
            
            self.pool = nn.MaxPool2d(2, 2)
        # default stride is 2 because it was not specified so defaults to kernel size which is 2
        # output size = ((n-f+1)/2)^2 = 22*22 *6  
            
            self.conv2 = nn.Conv2d(6, 16, 5)
        #output size = 18 * 18 * 16 = 5184   
            
            self.bn2 = nn.BatchNorm2d(16)  # Batch normalization after conv2
            
            self.fc1 = nn.Linear(16 * 9 * 9, 120)
            self.fc2 = nn.Linear(120, 84)
            self.fc3 = nn.Linear(84, 10)
            
            self.dropout = nn.Dropout(p=drop)
            
        def forward(self, x):
            x = self.pool(activation(self.bn1(self.conv1(x)))) 
            # 44*44*6 , 22*22*6 
            
            x = self.pool(activation(self.bn2(self.conv2(x))))
            # 18*18*16 , 9*9*16 
            
            x = torch.flatten(x, 1) # flatten all dimensions except batch
            x = activation(self.dropout(self.fc1(x)))
            # x = self.dropout(x)
            x = activation(self.dropout(self.fc2(x)))
            # x = self.dropout(x)
            x = self.fc3(x)
            return x

In [13]:
torch.manual_seed(42)
torch.mps.manual_seed(42) #do I need it ? wasn't in the current stats
model = Net()
# model.load_state_dict()
model.to(device)

print(model.state_dict)

<bound method Module.state_dict of Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=1296, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)>


In [14]:
train_and_validate(epochs, optimizer, scheduler , criterion, model, trainloader, validloader,learning_rate)

Epoch 1/20:
Training Loss: 16.7044 | Training Accuracy: 48.63%
Validation Loss: 15.0778 | Validation Accuracy: 56.85%
-----------------------------------
Epoch 2/20:
Training Loss: 14.8295 | Training Accuracy: 58.10%
Validation Loss: 13.5416 | Validation Accuracy: 62.49%
-----------------------------------
Epoch 3/20:
Training Loss: 14.0673 | Training Accuracy: 61.58%
Validation Loss: 13.0303 | Validation Accuracy: 64.98%
-----------------------------------
Epoch 4/20:
Training Loss: 13.6253 | Training Accuracy: 63.60%
Validation Loss: 13.4554 | Validation Accuracy: 61.79%
-----------------------------------
Epoch 5/20:
Training Loss: 13.3259 | Training Accuracy: 64.99%
Validation Loss: 12.4611 | Validation Accuracy: 67.69%
-----------------------------------
Epoch 6/20:
Training Loss: 13.0992 | Training Accuracy: 65.88%
Validation Loss: 12.0926 | Validation Accuracy: 69.87%
-----------------------------------
Epoch 7/20:
Training Loss: 12.9178 | Training Accuracy: 66.89%
Validation Lo

In [ ]:
# torch.save(model.state_dict(), './mymodel.pth')

In [ ]:
# dictionary for classess names in FERplus dataset
classes = {
    0: 'Neutral',
    1: 'Happinnes',
    2: 'Surprise',
    3: 'Sadness',
    4: 'Anger',
    5: 'Disgust',
    6: 'Fear',
    7: 'Contempt',
    8: 'Unknown',
    9: 'NF'
}

In [ ]:
np_img = test_dataset[88]['image']
plt.imshow(np.transpose(np_img, (1, 2, 0)))
print('image:\n')
plt.show()
plt.close()
print('truth:', classes[test_dataset[88]['emotions'].argmax(0)])

In [ ]:
my_model = Net()
my_model.load_state_dict(torch.load('./mymodel.pth'))
my_model.eval()

In [ ]:
from torchmetrics import Accuracy

torchmetrics_accuracy = Accuracy(task='multiclass', num_classes=10).to(device)

In [ ]:
# test model on the test data set
import torch.nn.functional as F
import torch
def test_model(model, test_loader, dataset):
    model.to('cpu')
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    test_accuracy = []
    wrong = 0
    with torch.inference_mode():
        for i, data in enumerate(test_loader):
            labels = data['emotions']
            inputs = data['image']
            output = model(inputs)
            # print("out: ", output.shape)
            # print("labels: ", labels.shape)
            # print(labels.shape)
            # print(f"test loader length:  {len(test_loader)}")
            # show the first 10 images and their truth labels and their predictions
            if i < len(test_loader):
                for j in range(len(inputs)):
                    # if classes[labels[j].argmax(0).item()] != classes[output[j].argmax(0).item()] :
                    #     wrong +=1
                    
                    
                    file_name = "mymodel.txt"
                    output_file = open(file_name, "a")
                    prediction = classes[labels[j].argmax(0).item()] == classes[output[j].argmax(0).item()]
                    
                    output_file.write(f"image: {i * batch_size + j} was {prediction}\n")
                    output_file.write(f"predicted label was: {classes[output[j].argmax(0).item()]} and the original label was {classes[labels[j].argmax(0).item()]}\n")
                    output_file.write("="*50)
                    output_file.write("\n")
                    # if j == 10:
                    #     print("one example:")
                    #     np_img = test_dataset[i * batch_size + j]['image']
                    #     plt.imshow(np.transpose(np_img, (1, 2, 0)))
                    #     plt.show()
                    #     plt.close()
                    #     print('-'*50)
                        
                    # print(labels[i].argmax().item())
                    # if classes[labels[j].argmax(0).item()] == classes[output[j].argmax(0).item()]:
                    #     print("hoppppaaaaaaaaaa")
                    #     print('image:', i * batch_size + j)
                        
                    #     print('truth:', classes[labels[j].argmax(0).item()])
                    #     print('prediction:', classes[output[j].argmax(0).item()])
                    #     print('='*50)
                    # plt.close('all')    
            output_file.close()
            # measure accuracy
            _, predicted = torch.max(output, 1)
            _, labels = torch.max(labels, 1)
            # torchmetrics_accuracy(predicted, labels)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            test_accuracy.append(torchmetrics_accuracy(predicted, labels))
        # print total accuracy
        print('Accuracy of the network on the test images: %d %%' % (
            100 * correct / total))
        print(f"correct: {correct}, total: {total}")
        print(test_accuracy)
test_model(my_model, testloader, test_dataset)

In [9]:
idx = 388

np_img = test_dataset[idx]['image']
plt.imshow(np.transpose(np_img, (1, 2, 0)))
plt.show()
plt.close()
print (f"the original label was {classes[test_dataset[idx]['emotions'].argmax(0).item()]}\n")


TypeError: Unexpected type <class 'numpy.ndarray'>

1- data augmentation 
2- maxing before ?
3- 

questions: 

1- how should the final product look like? 
2- should i investigate data augmentation? 
3- how about transfer learning? any advice? 
4- REPORT, how should it look like? any examples? any guidance is appreciated
5- in general, what do you think? 

genetic programming (evolutionary algorithms) 

leaky Relu and param Relu

regularization 

torch metrics 